## Hyperparam Tuning

Now that we know which models are performing better, it's time to perform cross validation and tune hyperparameters.
- Do a google search for hyperparameter ranges for each type of model.

GridSearch/RandomSearch are a great methods for checking off both of these tasks.

There is a fairly significant issue with this approach for this particular problem (described below). But in the interest of creating a basic functional pipeline, you can just use the default Sklearn methods for now.

## Preventing Data Leakage in Tuning - STRETCH

> **This step doesn't need to be part of your Minimum Viable Product (MVP), but its highly recommended you complete it, if you have time!**

BUT we have a problem - if we calculated a numerical value to encode city (such as the mean of sale prices in that city) on the training data, we can't cross validate 
- The rows in each validation fold were part of the original calculation of the mean for that city - that means we're leaking information!
- While sklearn's built in functions are extremely useful, sometimes it is necessary to do things ourselves

You need to create two functions to replicate what Gridsearch does under the hood. This is a challenging, real world data problem! To help you out, we've created some psuedocode and docstrings to get you started. 

**`custom_cross_validation()`**
- Should take the training data, and divide it into multiple train/validation splits. 
- Look into `sklearn.model_selection.KFold` to accomplish this - the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) shows how to split a dataframe and loop through the indexes of your split data. 
- Within your function, you should compute the city means on the training folds just like you did in Notebook 1 - you may have to re-join the city column to do this - and then join these values to the validation fold

This psuedocode may help you fill in the function:

```python
kfold = KFold() # fit sklearn k folds on X_train
train_folds = []
val_folds = []
for training_index, val_index in kfold.split(X_train):
    train_fold, val_fold = #.iloc loop variables on X_train

    # recompute training city means like you did in notebook 1 
    # merge to validation fold
        
    train_folds.append(train_fold)
    val_folds.append(val_fold)

    return train_folds, val_folds
```


**`hyperparameter_search()`**
- Should take the validation and training splits from your previous function, along with your dictionary of hyperparameter values
- For each set of hyperparameter values, fit your chosen model on each set of training folds, and take the average of your chosen scoring metric. [itertools.product()](https://docs.python.org/3/library/itertools.html) will be helpful for looping through all combinations of hyperparameter values
- Your function should output the hyperparameter values corresponding the highest average score across all folds. Alternatively, it could also output a model object fit on the full training dataset with these parameters.


This psuedocode may help you fill in the function:

```python
hyperparams = # Generate hyperparam options with itertools
hyperparam-scores = []
for hyperparam-combo in hyperparams:

    scores = []

    for folds in allmyfolds:
        # score fold the fold with the model/ hyperparams
        scores.append(score-fold)
        
    score = scores.mean()
    hyperparam-scores.append(score)
# After loop, find max of hyperparam-scores. Best params are at same index in `hyperparams` loop iteratble
```

Docstrings have been provided below to get you started. Once you're done developing your functions, you should move them to `functions_variables.py` to keep your notebook clean 

Bear in mind that these instructions are just one way to tackle this problem - the inputs and output formats don't need to be exactly as specified here.

In [8]:
# develop your custom functions here
import numpy as np
import itertools
from sklearn.model_selection import KFold
from sklearn.ensemble     import RandomForestRegressor
from sklearn.metrics      import mean_squared_error

def custom_cross_validation(training_data, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    train_folds, val_folds = [], []
    for train_idx, val_idx in kf.split(training_data):
        train_folds.append(training_data.iloc[train_idx])
        val_folds.append(  training_data.iloc[val_idx])
    return train_folds, val_folds

def hyperparameter_search(training_folds, validation_folds, param_grid):
    # 1) Generate hyperparam combos
    param_names   = list(param_grid.keys())
    hyperparams   = list(itertools.product(*param_grid.values()))
    
    hyperparam_scores = []  # store avg RMSE for each combo

    # 2) Loop through each hyperparam combination
    for combo in hyperparams:
        params = dict(zip(param_names, combo))
        scores = []  # one RMSE per fold

        # 3) Loop through each fold
        for train_df, val_df in zip(training_folds, validation_folds):
            # assume last column is target
            target = train_df.columns[-1]
            X_tr, y_tr = train_df.drop(target, axis=1), train_df[target]
            X_va, y_va = val_df.drop(target, axis=1), val_df[target]

            model = RandomForestRegressor(random_state=42, **params)
            model.fit(X_tr, y_tr)
            preds = model.predict(X_va)

            rmse = np.sqrt(mean_squared_error(y_va, preds))
            scores.append(rmse)

        # average across folds
        avg_score = np.mean(scores)
        hyperparam_scores.append(avg_score)

    # 4) Find the index of the best (lowest) RMSE
    best_idx    = int(np.argmin(hyperparam_scores))
    best_params = dict(zip(param_names, hyperparams[best_idx]))

    return best_params



## Hyperparam Tuning

In [13]:
# perform tuning and cross validation here 
# using GridsearchCV/ RandomsearchCV (MVP)
# or your custom functions
# 0) Bring your train/test back into scope

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble      import RandomForestRegressor
from sklearn.metrics       import mean_squared_error
import joblib, os
import pandas as pd
from sklearn.impute import SimpleImputer

# 0) Load processed splits
X_train = pd.read_csv("data/processed/X_train_scaled.csv")
X_test  = pd.read_csv("data/processed/X_test_scaled.csv")
y_train = pd.read_csv("data/processed/y_train.csv").squeeze()
y_test  = pd.read_csv("data/processed/y_test.csv").squeeze()

# 1) Impute any remaining NaNs with the median (fit on train, transform both)
imp = SimpleImputer(strategy="median")
X_train_imp = imp.fit_transform(X_train)
X_test_imp  = imp.transform( X_test)

# 2) Define the grid 
param_grid = {
    "n_estimators":     [50, 100, 200],
    "max_depth":        [None, 10, 20],
    "min_samples_leaf": [1, 2, 4],
}

# 3) Set up GridSearchCV with built-in RMSE scoring 
grid = GridSearchCV(
    estimator = RandomForestRegressor(random_state=42),
    param_grid = param_grid,
    scoring   = "neg_root_mean_squared_error",  # <- no more make_scorer hack
    cv        = 5,
    n_jobs    = -1,
    verbose   = 1
)

# 4) Run the search 
grid.fit(X_train_imp, y_train)

# best_params_ is the dict of hyperparams with the *lowest* CV‐RMSE
best_params = grid.best_params_
best_cv_rmse = -grid.best_score_

print("🔍 Best params:", best_params)
print(f"🔍 Best CV RMSE: {best_cv_rmse:,.2f}")

# 5) Evaluate on the hold‐out test set 
test_preds = grid.predict(X_test_imp)
test_rmse  = mean_squared_error(y_test, test_preds, squared=False)
print(f"🏁 Final Test RMSE: {test_rmse:,.2f}")

# 6) Save the winning model 
os.makedirs("models", exist_ok=True)
joblib.dump(grid.best_estimator_, "models/best_random_forest.pkl")
print("✅ Model saved to models/best_random_forest.pkl")


Fitting 5 folds for each of 27 candidates, totalling 135 fits
🔍 Best params: {'max_depth': 20, 'min_samples_leaf': 2, 'n_estimators': 100}
🔍 Best CV RMSE: 410,120.31
🏁 Final Test RMSE: 525,311.80
✅ Model saved to models/best_random_forest.pkl


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


We want to make sure that we save our models.  In the old days, one just simply pickled (serialized) the model.  Now, however, certain model types have their own save format.  If the model is from sklearn, it can be pickled, if it's xgboost, for example, the newest format to save it in is JSON, but it can also be pickled.  It's a good idea to stay with the most current methods. 
- you may want to create a new `models/` subdirectory in your repo to stay organized

In [15]:
# save your best model here

import os, joblib

# 1. Create a models folder if it doesn’t exist
os.makedirs("models", exist_ok=True)

# 2. Save just the best estimator
best_model = grid.best_estimator_
joblib.dump(best_model, "models/best_random_forest.pkl")
print("✅ Saved model → models/best_random_forest.pkl")


✅ Saved model → models/best_random_forest.pkl


## Building a Pipeline (Stretch)

> **This step doesn't need to be part of your Minimum Viable Product (MVP), but its highly recommended you complete it if you have time!**

Once you've identified which model works the best, implement a prediction pipeline to make sure that you haven't leaked any data, and that the model could be easily deployed if desired.
- Your pipeline should load the data, process it, load your saved tuned model, and output a set of predictions
- Assume that the new data is in the same JSON format as your original data - you can use your original data to check that the pipeline works correctly
- Beware that a pipeline can only handle functions with fit and transform methods.
- Classes can be used to get around this, but now sklearn has a wrapper for user defined functions.
- You can develop your functions or classes in the notebook here, but once they are working, you should import them from `functions_variables.py` 

In [ ]:
# Build pipeline here

Pipelines come from sklearn.  When a pipeline is pickled, all of the information in the pipeline is stored with it.  For example, if we were deploying a model, and we had fit a scaler on the training data, we would want the same, already fitted scaling object to transform the new data with.  This is all stored when the pipeline is pickled.
- save your final pipeline in your `models/` folder

In [ ]:
# save your pipeline here